# Data Scientist

In [ ]:
import flwr

import syft_flwr

print(f"{syft_flwr.__version__=}")
print(f"{flwr.__version__=}")

In [ ]:
from pathlib import Path

SYFT_FLWR_PROJECT_DIR = Path("./quickstart-pandas")

assert SYFT_FLWR_PROJECT_DIR.exists()

In [ ]:
from pathlib import Path

from syft_rds.orchestra import setup_rds_server

DS = "ds@openmined.org"
DO1 = "do1@openmined.org"
DO2 = "do2@openmined.org"

ds_stack = setup_rds_server(email=DS, key="flwr", root_dir=Path("."))

In [ ]:
RUN_SIMULATION = 1

if RUN_SIMULATION:
    !flwr run {SYFT_FLWR_PROJECT_DIR}

In [ ]:
do_client_1 = ds_stack.init_session(host=DO1)
print("Logged into: ", do_client_1.host)

do_client_2 = ds_stack.init_session(host=DO2)
print("Logged into: ", do_client_2.host)

In [ ]:
SYFTBOX_DATASET_NAME = "iris"

dataset1 = do_client_1.dataset.get(name=SYFTBOX_DATASET_NAME)
dataset1.describe()

In [ ]:
SYFTBOX_DATASET_NAME = "iris"

dataset2 = do_client_2.dataset.get(name=SYFTBOX_DATASET_NAME)
dataset2.describe()

In [ ]:
import syft_flwr

try:
    !rm -rf {SYFT_FLWR_PROJECT_DIR / "main.py"}
    syft_flwr.bootstrap(SYFT_FLWR_PROJECT_DIR, aggregator=DS, datasites=[DO1, DO2])
    print("Bootstrapped project successfully ✅")
except Exception as e:
    print(f"Bootstrapped project failed with error: '{e}' ❌")

In [ ]:
if RUN_SIMULATION:
    syft_flwr.run(
        SYFT_FLWR_PROJECT_DIR, [dataset1.get_mock_path(), dataset2.get_mock_path()]
    )

In [ ]:
!rm -rf {SYFT_FLWR_PROJECT_DIR / "pandas_example" / "__pycache__/"}
!rm -rf {SYFT_FLWR_PROJECT_DIR / "simulation_logs"}

In [ ]:
# Job submission
datasites = [do_client_1, do_client_2]

for client in datasites:
    job = client.jobs.submit(
        name="iris_fed_analytics",
        description="Syft Flower Federated Analytics on the Iris Dataset",
        user_code_path=SYFT_FLWR_PROJECT_DIR,
        dataset_name=SYFTBOX_DATASET_NAME,
        tags=["federated", "analytics", "syft_flwr", "flwr"],
        entrypoint="main.py",
    )
    print(job)

In [ ]:
import os

os.environ["SYFTBOX_CLIENT_CONFIG_PATH"] = str(ds_stack.client.config_path)
os.environ["LOGURU_LEVEL"] = "DEBUG"

!uv run {str(SYFT_FLWR_PROJECT_DIR / "main.py")} --active